In [23]:
# Imports
import math
import numpy  as np
import pandas as pd
import random
import pickle
import warnings
import inflection
import seaborn as sns

from scipy                 import stats  as ss
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML


from sklearn.metrics       import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from src.tools             import ml_error
from src.tools             import timeSeries_CV
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from scipy.stats import randint

warnings.filterwarnings( 'ignore' )


In [7]:
X = pd.read_csv('data/X.csv')
y  = pd.read_csv('data/y.csv')

128

### 8. Hyperparameter Fine tuning

In [29]:

# Definir o modelo a ser otimizado (Regressão com Random Forest)
model = RandomForestRegressor(n_jobs= -1)

# Definir os hiperparâmetros e intervalos para otimização
param_dist = {
    'n_estimators': randint(100, 2000),
    'max_depth': randint(1, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Criar objeto TimeSeriesSplit para validação cruzada
tscv = TimeSeriesSplit(n_splits=5)

# Criar o objeto RandomizedSearchCV
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=25, cv=tscv, scoring='neg_mean_squared_error', random_state=42)


# Realizar a busca aleatória
search.fit(X, y)

# Avaliar o melhor modelo encontrado
best_model = search.best_estimator_

In [ ]:
# performance
rf_result_tuned = timeSeries_CV(X, y, model_name='Random Forest Regressor', model=best_model, kfold=5)
rf_result_tuned

,Model Name,MAE CV,MAPE CV,RMSE CV
0,Random Forest Regressor,1977.36 +/- 78.15,0.32 +/- 0.01,2812.44 +/- 125.31


In [ ]:
best_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 4,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 8,
 'min_samples_split': 8,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 221,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}